# Estudio de las estadísticas de un robot de limpieza reactivo

Dado:

Habitación de MxN espacios.
Número de agentes.
Porcentaje de celdas inicialmente sucias.
Tiempo máximo de ejecución.
Realiza la siguiente simulación:

Inicializa las celdas sucias (ubicaciones aleatorias).
Todos los agentes empiezan en la celda [1,1].
En cada paso de tiempo:
Si la celda está sucia, entonces aspira.
Si la celda está limpia, el agente elije una dirección aleatoria para moverse (unas de las 8 celdas vecinas) y elije la acción de movimiento (si no puede moverse allí, permanecerá en la misma celda).
Se ejecuta el tiempo máximo establecido.
Para un espacio de 100x100, considera los siguientes escenarios:

Escenario 1: 1 agente, 90% de celdas sucias.
Escenario 2. 2 agentes, 90% de celdas sucias.
Deberás resolver las siguientes preguntas:

¿Cuántos pasos de simulación toma limpiar todo el espacio?
¿Qué porcentaje de celdas sucias queda con los siguientes pasos de simulación: 100, 1000, 10000?
A continuación, determina cuál es la cantidad óptima de aspiradoras que debe de tener para realizar la limpieza en el menor tiempo posible. Considera que tenemos un máximo de 10 aspiradoras disponibles.

Desarrollar un informe con lo observado.

In [1]:
from mesa import Agent, Model

from mesa.space import MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation
plt.rcParams["animation.html"] = "jshtml"
matplotlib.rcParams['animation.embed_limit'] = 2**128

import numpy as np
import pandas as pd

import time
import datetime

In [ ]:
class VaccumAgent(Agent):
    def __init__ (self, id, model, x, y):
        super().__init__(id, model)
        self.pos = (x,y)

    def step(self):
        x_pos = self.pos[0]
        y_pos = self.pos[1]

        # Check if the current cell is dirty
        if self.model.grid[x_pos][y_pos] == 1:
            self.model.grid[x_pos][y_pos] = 0
            self.model.cleaned_cells += 1
            self.model.total_dirt -= 1
        
        # Move to a random cell
        possible_steps = self.model.grid.get_neighborhood(self.pos, moore=True, include_center=False)
        new_position = self.random.choice(possible_steps)
        cellmate = self.model.grid.get_cell_list_contents([new_position])

        self.pos = new_position
        self.model.grid.move_agent(self, new_position)

In [ ]:
def get_vaccums(model):
    return np.asarray([agent.pos for agent in model.schedule.agents])

In [ ]:
class VaccumModel(Model):
    def __init__ (self, width, height, num_agents, dirty_percentage):
        self.grid = MultiGrid(width, height, torus = False)
        self.schedule = RandomActivation(self)
        self.datacollector = DataCollector(
            model_reporters = {"Vaccums": get_vaccums},
            agent_reporters = {"Position": "pos"}
        )
        self.dirty_cells = np.zeros((width, height))
        self.dirty_percentage = dirty_percentage
        self.running = True

        # Create agents
        for i in range(num_agents):
            x = 1
            y = 1
            agent = VaccumAgent(i, self, x, y)
            self.schedule.add(agent)
            self.grid.place_agent(agent, (x,y))

        # Create dirty cells
        dirt_cells = 0
        while total_dirt < int(width * height * (dirty_percentage / 100)):
            x = int(np.random.rand() * width)
            y = int(np.random.rand() * height)
            if self.dirty_cells[x][y] == 0:
                self.dirty_cells[x][y] = 1
                dirt_cells += 1

        def step(self):
            self.datacollector.collect(self)
            self.schedule.step()


In [ ]:
WIDTH = 100
HEIGHT = 100
DIRTY_PERCENTAGE = 90
ITERATIONS = [100,1000,10000]

In [ ]:
def clean(model, iterations, num_vaccums):
    print ("Number of vaccums: ", num_vaccums)

    for iteration in interations:
        time = time.time()

        print("Iteration: ", iteration)

        for i in range(interation):
            model.step()

            if model.running == False:
                break

        dirty_cells = model.dirty_cells.sum()
        dirty_percentage = (dirty_cells / (WIDTH * HEIGHT)) * 100

        print("Dirty cells: ", dirty_cells)
        print("Time: ", time.time() - time)

In [ ]:
def clean_all_cells(model, num_vaccums):
    time = time.time()
    steps = 0

    print("Agents: ", num_vaccums)
    
    while model.running == True:
        model.step()
        steps += 1

    print("Steps: ", steps)

    total_time = time.time() - time
    print("Total time: ", total_time)

    return [total_time, steps]

In [ ]:
NUM_VACCUMS = 1
model = VaccumModel(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_PERCENTAGE)
clean_all_cells(model, NUM_VACCUMS)

model = VaccumModel(WIDTH, HEIGHT, NUM_VACCUMS, DIRTY_PERCENTAGE)
clean(model, ITERATIONS, NUM_VACCUMS)